# Convolutional gridding

We demonstrate how the convolutional gridding technique works with a little animation.

We start by adjusting the notebook settings.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from astropy.wcs import WCS
import cygrid

np.seterr(divide='ignore', invalid='ignore')
rc('animation', html='html5')

For our first demonstration, we will sample a pre-defined function at random position (for longitude and latitude between -1 and 1) and grid to a regular map. While we do this, we display intermediary results.

In [3]:
def func(l, b):
    return l * (1 - l) * np.cos(4 * np.pi * l) * np.sin(4 * np.pi * b ** 2) ** 2

In [4]:
header = {
    'NAXIS': 3,
    'NAXIS1': 101,
    'NAXIS2': 101,
    'NAXIS3': 1,  # need dummy spectral axis
    'CTYPE1': 'GLON-CAR',
    'CTYPE2': 'GLAT-CAR',
    'CUNIT1': 'deg',
    'CUNIT2': 'deg',
    'CDELT1': -2 / 101,
    'CDELT2': 2 / 101,
    'CRPIX1': 51,
    'CRPIX2': 51,
    'CRVAL1': 0,
    'CRVAL2': 0,
    }

In [5]:
def sample(nsize):
    
    l, b = np.random.uniform(-1, 1, (2, nsize))
    signal = func(l, b)
    
    return l, b, signal

In [6]:
nsize = 51000
all_l, all_b, all_signal = sample(nsize)

In [7]:
l, b, signal = all_l[0:1], all_b[0:1], all_signal[0:1]

In [8]:
gridder = cygrid.WcsGrid(header)

kernelsize_fwhm = 3 / 60  # in degrees
kernelsize_sigma = kernelsize_fwhm / np.sqrt(8 * np.log(2))
kernel_support = 4 * kernelsize_sigma

gridder.set_kernel(
    'gauss1d',
    (kernelsize_sigma,),
    kernel_support,
    kernel_support / 2.
    )

In [9]:
gridder.grid(l, b, signal[:, np.newaxis])

In [10]:
slice_list = (
    list(range(0, 20, 1)) +
    list(range(20, 200, 10)) +
    list(range(200, 2000, 100)) +
    list(range(2000, 31001, 1000))
    )
start_i = slice_list[:-1]
end_i = slice_list[1:]

In [11]:
fig = plt.figure(figsize=(12, 12))
axes = [
    fig.add_subplot(2, 2, idx, projection=WCS(header).celestial)
    for idx in range(1, 5)
    ]
imkw = dict(origin='lower', interpolation='nearest', cmap='viridis')

tsignal = np.full_like(all_signal, np.nan)
points = axes[0].scatter(
    all_l, all_b, c=all_signal, alpha=0,
    cmap='viridis', vmin=-0.5, vmax=0.5
    )
title_p = axes[0].set_title('Raw samples (iter: {:d})'.format(0), loc='center', fontsize=14)
im1 = axes[1].imshow(gridder.get_datacube()[0], **imkw, vmin=-0.5, vmax=0.5)
title1 = axes[1].set_title('Gridded data (iter: {:d})'.format(0), loc='center', fontsize=14)
im2 = axes[2].imshow(gridder.get_unweighted_datacube()[0], **imkw, vmin=-10, vmax=10)
title2 = axes[2].set_title('Gridded data * weight map (iter: {:d})'.format(0), loc='center', fontsize=14)
im3 = axes[3].imshow(gridder.get_weights()[0], **imkw, vmin=0, vmax=10)
title3 = axes[3].set_title('Weight map (iter: {:d})'.format(0), loc='center', fontsize=14)



def init():
    face_colors = points.get_facecolors()
    face_colors[0, 3] = 1
    im1.set_array(gridder.get_datacube()[0])
    im2.set_array(gridder.get_unweighted_datacube()[0])
    im3.set_array(gridder.get_weights()[0])
    # title.set_text('0')
    return (
        points, im1, im2, im3,
        title_p, title1, title2, title3
        )

def animate(i):
    _sl = slice(start_i[i], end_i[i])
    l, b, signal = all_l[_sl], all_b[_sl], all_signal[_sl]
    gridder.grid(l, b, signal[:, np.newaxis])
    
    face_colors = points.get_facecolors()
    face_colors[_sl, 3] = 1
    im1.set_array(gridder.get_datacube()[0])
    im2.set_array(gridder.get_unweighted_datacube()[0])
    im3.set_array(gridder.get_weights()[0])
    title_p.set_text('Raw samples (iter: {:d})'.format(start_i[i]))
    title1.set_text('Gridded data (iter: {:d})'.format(start_i[i]))
    title2.set_text('Gridded data * weight map (iter: {:d})'.format(start_i[i]))
    title3.set_text('Weight map (iter: {:d})'.format(start_i[i]))
    return (
        points, im1, im2, im3,
        title_p, title1, title2, title3
        )

anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=len(start_i), interval=200, blit=True
    )

# this takes a while!
plt.close(anim._fig)
anim